# California Crime Analysis (R)
### Dataset Description
This project uses the “California Crime and Law Enforcement” dataset (2015) from Kaggle. It includes crime statistics by city and county in California (e.g., violent and property crimes) as well as corresponding law enforcement employee numbers. The dataset enables analysis of crime distribution, officer-to-population ratios, outlier detection, and exploratory modeling.


In [5]:
#Libraries
library(readr)

In [7]:
law_city <- read.csv("C:/Users/chris/OneDrive/Έγγραφα/R_scripts/ca_law_enforcement_by_city.csv")
head(law_city)

,City,Population,Total.law.enforcement.employees,Total..officers,Total..civilians
,<chr>,<chr>,<chr>,<chr>,<chr>
1,Alameda,"78,613",112,83,29
2,Albany,"19,723",30,23,7
3,Alhambra,"86,175",128,85,43
4,Alturas,"2,566",6,5,1
5,Anaheim,"349,471",577,399,178
6,Anderson,"10,277",24,18,6
